In [24]:
import startup
import numpy as np
import os
import time
import io
import tensorflow as tf
from models import models
import matplotlib
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from util.system import setup_environment
from util.train import get_trainable_variables, get_learning_rate_origin, get_learning_rate, get_path
from util.losses import regularization_loss
from util.fs import mkdir_if_missing
from util.data import tf_record_compression
import tensorflow.contrib.slim as slim
from scipy.io import loadmat
tfsum = tf.contrib.summary

no display found. Using non-interactive Agg backend


In [25]:
def parse_tf_records(cfg, serialized):
    num_views = cfg.num_views
    image_size = cfg.image_size

    # A dictionary from TF-Example keys to tf.FixedLenFeature instance.
    features = {
        'name': tf.FixedLenFeature([1], tf.string),
        'image': tf.FixedLenFeature([num_views, image_size, image_size, 3], tf.float32),
        'mask': tf.FixedLenFeature([num_views, image_size, image_size, 1], tf.float32),
        'inpoints':tf.FixedLenFeature([num_views, cfg.gt_point_n, 2], tf.float32),
    }

    if cfg.saved_camera:
        features.update(
            {'extrinsic': tf.FixedLenFeature([num_views, 4, 4], tf.float32),
             'cam_pos': tf.FixedLenFeature([num_views, 3], tf.float32)})
    if cfg.saved_depth:
        features.update(
            {'depth': tf.FixedLenFeature([num_views, image_size, image_size, 1], tf.float32)})

    return tf.parse_single_example(serialized, features)

In [26]:
def gen_plot(x, y, title):
    """Create a pyplot plot and save to buffer."""
    plt.figure()
    plt.plot(x, y)
    print(x.shape, y.shape)
    plt.title(title)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return buf

In [27]:
from util.config import merge_configs_recursive

In [28]:
cfg = merge_configs_recursive([{}])

In [29]:
train_dir = get_path(cfg)
train_dir = os.path.join(train_dir, str(cfg.vox_size))

In [30]:
tf.logging.set_verbosity(tf.logging.INFO)
split_name = 'train'
dataset_file = os.path.join("../../../../Datasets/2Dpm/tf_records_new/", f"{cfg['synth_set']}_{split_name}.tfrecords")
print(dataset_file)

dataset = tf.data.TFRecordDataset(dataset_file, compression_type=tf_record_compression(cfg))
dataset = dataset.map(lambda rec: parse_tf_records(cfg, rec), num_parallel_calls=4) \
        .batch(cfg.batch_size) \
        .prefetch(buffer_size=100) \

../../../../Datasets/2Dpm/tf_records_new/03001627_train.tfrecords


In [31]:
iterator = dataset.make_one_shot_iterator()
train_data = iterator.get_next()

In [33]:
train_data['name']

<tf.Tensor: id=88, shape=(1, 1), dtype=string, numpy=array([[b'7fe64a3a70f8f6b28cd4e3ad2fcaf039']], dtype=object)>

In [10]:
global_step = tf.train.get_or_create_global_step()
model = models.ModelPointCloud(cfg, global_step)
inputs = model.preprocess(train_data, cfg.step_size)

In [12]:
max_to_keep = 120
saver = tf.train.Saver(max_to_keep=max_to_keep)

session_config = tf.ConfigProto(
    log_device_placement=False)
session_config.gpu_options.allow_growth = cfg.gpu_allow_growth
session_config.gpu_options.per_process_gpu_memory_fraction = cfg.per_process_gpu_memory_fraction

In [16]:
with tf.Session(config=session_config) as sess:
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()
        summary_writer = tf.summary.FileWriter(train_dir, flush_secs=10, graph=sess.graph)
        count = 0
        while count <= 5:
            images = sess.run([inputs['images']])[0]
            print(images.shape)
            count += 1

(4, 128, 128, 3)
(4, 128, 128, 3)
(4, 128, 128, 3)
(4, 128, 128, 3)
(4, 128, 128, 3)
(4, 128, 128, 3)


In [8]:




model_fn = model.get_model_fn(
    is_training=True, reuse=False, run_projection=True)

outputs = model_fn(inputs)

# train_scopes
train_scopes = ['encoder', 'decoder']

task_loss, c_loss, k_loss, de_loss = model.get_loss(inputs, outputs)
reg_loss = regularization_loss(train_scopes, cfg)
loss = task_loss

learning_rate = get_learning_rate(cfg, global_step)
tf.summary.scalar("Learning_Rate", learning_rate)
# Merge all Summaries
summary_op = tf.summary.merge_all()


var_list = get_trainable_variables(train_scopes)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss, global_step, var_list)

# Epoch Loss summary
tf.summary.scalar("Epoch_Loss", per_epoch_loss)
loss_summary_op = tf.summary.merge_all()

pred_image = tf.image.decode_png(pred_buf.getvalue(), channels=4)
gt_image = tf.image.decode_png(gt_buf.getvalue(), channels=4)
tf.summary.image("Predicted_Image", pred_image)
tf.summary.image("Target_Image", gt_image)
im_summary_op = tf.summary.merge_all()

(4, 40, 40, 3) (4, 512)
(1, 2048) (1, 40, 40, 12)
Instructions for updating:
dim is deprecated, use axis instead
[<tf.Variable 'encoder/Conv/weights:0' shape=(5, 5, 3, 16) dtype=float32_ref>, <tf.Variable 'encoder/Conv/batch_normalization/gamma:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'encoder/Conv/batch_normalization/beta:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'encoder/Conv_1/weights:0' shape=(3, 3, 16, 32) dtype=float32_ref>, <tf.Variable 'encoder/Conv_1/batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'encoder/Conv_1/batch_normalization/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'encoder/Conv_2/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'encoder/Conv_2/batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'encoder/Conv_2/batch_normalization/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'encoder/Conv_3/weights:0' shape=(3, 3, 32, 64) dtype=float32_ref>, <tf.Variable 'encoder/Conv_3/bat

AttributeError: 'Tensor' object has no attribute 'getvalue'

In [9]:
with tf.Session(config=session_config) as sess:
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()
        summary_writer = tf.summary.FileWriter(train_dir, flush_secs=10, graph=sess.graph)
        global_step_val = 0
        epoch_loss = 0
        t0 = time.perf_counter()
        while global_step_val <= cfg['max_number_of_steps']:
            _, loss_val, global_step_val, summary, result, gtpoints = sess.run([train_op, loss, global_step, summary_op, outputs, inputs['inpoints']])
            print(summary)
            summary_writer.add_summary(summary, global_step_val)
            temp = result['all_points']
            points3d = result['points3D']
            assert temp[0].all() == temp[1].all()
            assert temp[0].all() == points3d.all()
            is_nan = np.isnan(loss_val)
            assert(not np.any(is_nan))
            epoch_loss += loss_val
            
            if global_step_val % 2 == 0 and global_step_val > 0:
                pred = result['test_o'][0]
                gt = gtpoints[0]
                
                predx = np.reshape(pred[:, 0], [pred.shape[0], 1])
                predy = np.reshape(pred[:, 1], [pred.shape[0], 1])

                gtx = np.reshape(gt[:, 0], [gt.shape[0], 1])
                gty = np.reshape(gt[:, 1], [gt.shape[0], 1])

                pred_buf = gen_plot(predx, predy, "Predicted Data")
                gt_buf = gen_plot(gtx, gty, "Target Image")
                
                im_summary = sess.run(im_summary_op, feed_dict = {pred_buf: pred_buf, gt_buf: gt_buf})
                print(im_summary)
                summary_writer.add_summary(im_summary, global_step_val)

NameError: name 'session_config' is not defined

In [ ]:
pwd